In [1]:
import sys
sys.path.append('../FaaSterConfig')
import subprocess
import time
import os
import select
import datetime
import re
import requests

from FaaSterConfig import * 
sys.argv = './FaaSterConfig.py ../openFaas/matmul.yml -c 1 2 -m 248 1024 -nt NoGPU -v'.split(" ")

In [2]:
args = parseArgs()
doe, stack = generateFunctionConfigs(**args)
# writeStack(stack, args['genStackPath'])
doe

finalArgs: {'stackPath': '../openFaas/matmul.yml', 'funcName': 'matmul', 'outStack': '../openFaas/matmul_gen.yml', 'cpu': [1.0, 2.0], 'mem': [248.0, 1024.0], 'nodeType': ['NoGPU'], 'data': '1000', 'verbose': True, 'local': None, 'timeout': None, 'genStackPath': '../openFaas/matmul_gen.yml', 'config_space': {'CPU': [1.0, 2.0], 'Mem': [248.0, 1024.0], 'NodeType': ['NoGPU']}, 'url': 'http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080'}


/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NoGPU' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]


,CPU,Mem,NodeType,i,NodeTypeStr,funcName
0,1.0,248.0,0.0,1,NoGPU,matmul-cpu1x0-mem-248x0-nogpu
1,2.0,248.0,0.0,2,NoGPU,matmul-cpu2x0-mem-248x0-nogpu
2,1.0,1024.0,0.0,3,NoGPU,matmul-cpu1x0-mem-1024x0-nogpu
3,2.0,1024.0,0.0,4,NoGPU,matmul-cpu2x0-mem-1024x0-nogpu


In [ ]:
def imageExists(image):
    cmd = f"docker manifest inspect {image} > /dev/null ; echo $?"
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    (out, err) = process.communicate()
    if err:
        print(err)
    return out.strip() == "0"

def imagesExist(stackPath):
    stack = readStack(stackPath)
    return all(map(imageExists, set(v['image'] for k,v in stack['functions'].items())))

def up(stackYml, verbose=False, **kwargs):
    subcmd = "deploy" if imagesExist(stackYml) else "up"
    cmd = f"cd {os.path.dirname(stackYml)}; faas-cli {subcmd} -f {stackYml}"
    if verbose: 
        print(f"running command: {cmd}")
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    result = None
    for line in process.stdout:
        if verbose:
            print(line)
        if match:=re.search("Deployed. (\d+) Accepted.", line):
            result = f"Deployed {match.groups()[0]}"
        if match:=re.search("failed to deploy", line):
            raise Exception(f"Failed: {line}")
        
    return result

def remove(stackPath, verbose=False):
    cmd = f"cd {os.path.dirname(stackPath)}; faas-cli rm -f {stackPath}"
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    for line in process.stdout:
        if verbose:
            print(line)

    return True

In [3]:
def remoteMainJupyter(args):
    doe, stack = generateFunctionConfigs(**args)
    writeStack(stack, args['genStackPath'])
    print(f"Generated {len(doe)} configurations, testing on remote now")
    up(args['genStackPath'], **args)
    
    doe = getTimes(doe, **args)
    doe.sort_values(by=["time"], inplace=True)
    print(tabulate(doe[['CPU','Mem','NodeTypeStr', 'time']].reset_index(drop=True), headers='keys', tablefmt='psql'))

    best = doe.iloc[0]
    print(f"Top Recommendation config: CPU :{best.CPU}, Mem: {best.Mem}, NodeType: {best.NodeTypeStr} which had a final time of: {best.time}s")
    return args, doe, stack

In [3]:
args, doe, stack = main()

finalArgs: {'stackPath': '../openFaas/matmul.yml', 'funcName': 'matmul', 'outStack': '../openFaas/matmul_gen.yml', 'cpu': [1.0, 2.0], 'mem': [248.0, 1024.0], 'nodeType': ['NoGPU'], 'data': '1000', 'verbose': True, 'local': None, 'timeout': None, 'genStackPath': '../openFaas/matmul_gen.yml', 'config_space': {'CPU': [1.0, 2.0], 'Mem': [248.0, 1024.0], 'NodeType': ['NoGPU']}, 'url': 'http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080'}
Generated 4 configurations, testing on remote now


/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NoGPU' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]


running command: cd ../openFaas; faas-cli deploy -f ../openFaas/matmul_gen.yml
Deploying: matmul-cpu2x0-mem-1024x0-nogpu.

WARNING! You are not using an encrypted connection to the gateway, consider using HTTPS.



Deployed. 202 Accepted.

URL: http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080/function/matmul-cpu2x0-mem-1024x0-nogpu



Deploying: matmul-cpu2x0-mem-248x0-nogpu.

WARNING! You are not using an encrypted connection to the gateway, consider using HTTPS.



Deployed. 202 Accepted.

URL: http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080/function/matmul-cpu2x0-mem-248x0-nogpu



Deploying: matmul-cpu1x0-mem-1024x0-nogpu.

WARNING! You are not using an encrypted connection to the gateway, consider using HTTPS.



Deployed. 202 Accepted.

URL: http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080/function/matmul-cpu1x0-mem-1024x0-nogpu



Deploying: matmul-cpu1x0-m

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
doe

In [8]:
def runFunction(funcName=None, baseUrl=None, data="100", verbose=False, **kwargs):
    assert funcName is not None, "funcName required"
    assert baseUrl is not None, "baseUrl required"
    # TODO get time from faas-cli logs instead of response.elapsed
    if verbose:
        print(datetime.datetime.now())
    
    url = f"{baseUrl}/function/{funcName}"
    response = requests.post(url, data=data, timeout=kwargs.get('timeout', 60))

    print(response)
    if not response.ok:
        print(response.content)
        return None
    
    return response.

In [12]:
response = runFunction("matmul-cpu1x0-mem-1024x0-nogpu", "http://k8s-openfaas-gatewaye-67e167a914-178e01451d9f2717.elb.us-east-1.amazonaws.com:8080", data="1000000000")

<Response [500]>
b'exit status 1\nTraceback (most recent call last):\n  File "/home/app/index.py", line 19, in <module>\n    ret = handler.handle(st)\n          ^^^^^^^^^^^^^^^^^^\n  File "/home/app/function/handler.py", line 20, in handle\n    result = matmul(n)\n             ^^^^^^^^^\n  File "/home/app/function/handler.py", line 5, in matmul\n    A = np.random.rand(n, n)\n        ^^^^^^^^^^^^^^^^^^^^\n  File "numpy/random/mtrand.pyx", line 1218, in numpy.random.mtrand.RandomState.rand\n  File "numpy/random/mtrand.pyx", line 436, in numpy.random.mtrand.RandomState.random_sample\n  File "_common.pyx", line 307, in numpy.random._common.double_fill\nnumpy.core._exceptions._ArrayMemoryError: Unable to allocate 6.94 EiB for an array with shape (1000000000, 1000000000) and data type float64\n'


In [13]:
response.content

AttributeError: 'NoneType' object has no attribute 'content'

In [14]:
'./FaaSterConfig.py ../openFaas/matmul.yml -c 1 2 -m 248 1024 -nt NoGPU -v'.split(" ")

['./FaaSterConfig.py',
 '../openFaas/matmul.yml',
 '-c',
 '1',
 '2',
 '-m',
 '248',
 '1024',
 '-nt',
 'NoGPU',
 '-v']